**Author:** Benoît BAILLIF

**Purpose:** Compute models

**Input:**
- data/processed/
 - cmp_info_cmap.csv : info on used compounds
 - used_pert_id_target_matrix.csv : find targets for each used compound
 - signatures/
 
**Output:** 
- tables for models

In [1]:
import os
import time
import numpy             as np
import pandas            as pd
import matplotlib.pyplot as plt
from pathlib import Path
from rdkit           import Chem
from rdkit.Chem      import AllChem
from sklearn                     import metrics
from sklearn.ensemble            import RandomForestClassifier
from sklearn.model_selection     import GridSearchCV
from sklearn.model_selection     import train_test_split
from sklearn.tree                import export_graphviz

In [2]:
SEED = 42
cell_lines = ['MCF7', 'HA1E', 'HT29', 'A549', 'HCC515', 'PC3', 'VCAP', 'A375']

# Input

In [3]:
processed_data_directory = 'data/processed/'
sigs_directory = processed_data_directory + 'signatures/'

In [4]:
cmp_info_cmap_path = processed_data_directory + 'cmp_info_cmap.csv'
used_pert_id_target_matrix_path = processed_data_directory + 'used_pert_id_target_matrix.csv'

In [5]:
performances_directory = 'performances/'
Path(performances_directory).mkdir(exist_ok=True)

# Load data, prepare models and variables to stock results

In [6]:
cmp_info_cmap = pd.read_csv(cmp_info_cmap_path, index_col='pert_id')
print(cmp_info_cmap.shape)
cmp_info_cmap.head()

(21220, 6)


,pert_iname,is_touchstone,inchi_key,canonical_smiles,pubchem_cid,used_compound
pert_id,,,,,,
BRD-A00100033,nifurtimox,1.0,ARFHIAQFJWUCFH-UHFFFAOYSA-N,CC1CS(=O)(=O)CCN1N=Cc1ccc([N+](=O)[O-])o1,6842999.0,1
BRD-A00150179,5-hydroxytryptophan,0.0,QSHLMQDRPXXYEE-UHFFFAOYSA-N,NC(Cc1c[nH]c2cccc(O)c12)C(=O)O,589768.0,0
BRD-A00267231,hemado,1.0,KOCIMZNSNPOGOP-UHFFFAOYSA-N,CCCCC#Cc1nc(NC)c2ncn(C3OC(CO)C(O)C3O)c2n1,4043357.0,1
BRD-A00420644,SA-3676,0.0,ASCBUEVCEVGOFP-UHFFFAOYSA-N,CCN1c2ccccc2NC2N=C(OC)C(c3ccccc3)C21,2853908.0,1
BRD-A00474148,BRD-A00474148,0.0,RCGAUPRLRFZAMS-UHFFFAOYSA-N,O=C1Cc2cc([S+](=O)([O-])N3CCN(c4ccc(O)cc4)CC3)...,44825297.0,1


In [7]:
used_cmp_info_cmap = cmp_info_cmap[cmp_info_cmap['used_compound'] == True]

In [8]:
used_pert_id_target_matrix = pd.read_csv(used_pert_id_target_matrix_path, index_col='pert_id')
print(used_pert_id_target_matrix.shape)
used_pert_id_target_matrix.head()

(7825, 1358)


,AAK1,ABAT,ABCA1,ABCB1,ABCB11,ABCB6,ABCC1,ABCC2,ABCC3,ABCC5,...,WEE2,WNT3A,WRN,XDH,XIAP,XPO1,YES1,YWHAB,YWHAG,ZAP70
pert_id,,,,,,,,,,,,,,,,,,,,,
BRD-A00100033,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BRD-A00218260,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BRD-A00267231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BRD-A00420644,NaN,NaN,NaN,0.0,NaN,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN
BRD-A00474148,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
used_pert_id_target_matrix['ABL1']

pert_id
BRD-A00100033   NaN
BRD-A00218260   NaN
BRD-A00267231   NaN
BRD-A00420644   NaN
BRD-A00474148   NaN
                 ..
BRD-K30126819   NaN
BRD-A05821830   NaN
BRD-K78567475   NaN
BRD-K68164687   NaN
BRD-K74057757   NaN
Name: ABL1, Length: 7825, dtype: float64

In [10]:
d_sigs = {}
for cell_line in cell_lines :
    sigs_df = pd.read_csv(sigs_directory + cell_line + '_used_signatures.csv', index_col='pert_id')
    sigs_df.columns = sigs_df.columns.str.lower() # to distinguich with target names
    d_sigs[cell_line] = sigs_df.copy()

In [11]:
targets = []
for c in used_pert_id_target_matrix.columns :
    n_actives = used_pert_id_target_matrix[c].sum()
    if n_actives > 50 :
        targets.append(c)
print('There are ' + str(len(targets)) + ' targets having at least 50 actives in our restricted dataset')

There are 95 targets having at least 50 actives in our restricted dataset


In [12]:
molecules = used_cmp_info_cmap['canonical_smiles'].apply(Chem.MolFromSmiles)
binary_fps = np.array([list(AllChem.GetMorganFingerprintAsBitVect(mol, radius=3, nBits=1024)) for mol in molecules])
bin_fp_df = pd.DataFrame(binary_fps, index=molecules.index)

# Target workflow

In [13]:
param_grid = {'max_depth' : np.arange(2,11,1)}
cv_fold = 4

In [14]:
def check_data_number(y, min_number_class=20, min_number_total=100) :
    
    # We need to prove that we have enough data, default value is False
    enough_data = False
    
    summary_series = pd.Series(dtype='float32')
    
    class_counts = y.value_counts()
    
    if 0 in class_counts :
        summary_series['Negative'] = class_counts[0]
    
    if 1 in class_counts :
        summary_series['Positive'] = class_counts[1]
    
    if len(class_counts) == 2 :
        positive_above_min = (class_counts[1] > min_number_class)
        negative_above_min = (class_counts[0] > min_number_class)
        total_above_min = (class_counts.sum() > min_number_total)
        if (positive_above_min & negative_above_min & total_above_min) :
            enough_data = True
    
    return enough_data, summary_series

In [15]:
def feature_selection(X, y, condition, n_selected_features=20) :
    
    global SEED
    
    fi_df = pd.DataFrame(index=X.columns)
    
    for i in np.arange(5) :
        rfclf = RandomForestClassifier(random_state=SEED, class_weight='balanced_subsample', n_estimators=200, n_jobs=-1).fit(X, y)
        feature_importances = rfclf.feature_importances_
        
        fi_df['fi_' + str(i)] = feature_importances
        
        SEED = SEED + 1
    
    ordered_fi_df = fi_df.mean(axis=1).sort_values(ascending=False)
    best_features = ordered_fi_df[:n_selected_features].index
    
    return best_features

In [16]:
def run_model(X, y, train_indices, test_indices, condition, summary_series) :

    X_train = X.iloc[train_indices,]
    X_test = X.iloc[test_indices,]
    y_train = y.iloc[train_indices,]
    y_test = y.iloc[test_indices,]
    
    # Feature selection
    best_features = feature_selection(X_train, y_train, condition)
    X_train = X_train[best_features]
    X_test = X_test[best_features]

    # CV
    rf = RandomForestClassifier(random_state=SEED, class_weight='balanced_subsample', 
                                            n_estimators=200)
    grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=cv_fold, verbose=1, scoring='roc_auc', n_jobs=-1)
    grid_search.fit(X_train, y_train)

    summary_series['ROC-AUC CV'] = grid_search.best_score_
    model = grid_search.best_estimator_

    print(grid_search.best_params_)
    
    # Test
    y_pred = model.predict(X_test)

    # Performances
    y_true = y_test
    
    tn, fp, fn, tp = metrics.confusion_matrix(y_true, y_pred).ravel()
    tn = float(tn)
    fp = float(fp)
    fn = float(fn)
    tp = float(tp)
    summary_series['TN'] = tn
    summary_series['FP'] = fp
    summary_series['FN'] = fn
    summary_series['TP'] = tp
        
    if (tp + fn) > 0 :
        summary_series['Sensitivity'] = tp / (tp + fn)
    if (tn + fp) > 0 :
        summary_series['Specificity'] = tn / (tn + fp)
        
    summary_series['Accuracy'] = metrics.accuracy_score(y_true, y_pred)
    summary_series['ROC-AUC test'] = metrics.roc_auc_score(y_true, y_pred)
        
    try :
        matthews = metrics.matthews_corrcoef(y_true, y_pred)
    except :
        print('Error in performance calculation')
    else :
        summary_series['Matthews'] = matthews
        
    #summary_series['Chosen features'] = X_train.columns
    
    return summary_series

In [17]:
def rf_workflow(X, y, train_indices, test_indices, summary_table, condition) :
    
    # Assess data sufficiency
    enough_data, summary_series = check_data_number(y)
    
    if enough_data :
        summary_series = run_model(X, y, train_indices, test_indices, condition, summary_series)
        summary_series = pd.DataFrame(summary_series).transpose()
        summary_series = summary_series.astype(float)
        info = condition.split('__')
        summary_series['target'] = info[0]
        summary_series['cell_line'] = info[1]
        summary_series['input'] = info[2]
        summary_series['iteration'] = info[3]
        summary_table = pd.concat([summary_table, summary_series])
    else :
        print('Not enough data for condition ' + condition)
        
    return summary_table

In [18]:
summary_table = pd.DataFrame()

time_start = time.time()

for i in np.arange(10) :

    for target in targets :

        for cell_line in cell_lines :
            
            print(target + '__' + cell_line + '__' + str(i))

            # Signature models
            cmp_tar = used_pert_id_target_matrix[target].dropna()
            cell_sigs = d_sigs[cell_line].copy()
            cell_sigs_tar = cell_sigs.join(cmp_tar, how='inner')

            n_samples = cell_sigs_tar.shape[0]
            indices = np.random.permutation(n_samples)
            limit = int(float(n_samples) * 0.67)
            train_indices, test_indices = indices[:limit], indices[limit:]

            X_sigs = cell_sigs_tar.drop(target, axis=1)
            y_sigs = cell_sigs_tar[target]

            condition = target + '__' + cell_line + '__sig__' + str(i)
            summary_table = rf_workflow(X_sigs, y_sigs, train_indices, test_indices, summary_table, condition)

            cell_fp = bin_fp_df[bin_fp_df.index.isin(cell_sigs_tar.index)]
            cell_fp_tar = cell_fp.join(cmp_tar, how='inner')

            X_fp = cell_fp_tar.drop(target, axis=1)
            y_fp = cell_fp_tar[target]

            condition = target + '__' + cell_line + '__binary_fp__' + str(i)
            summary_table = rf_workflow(X_fp, y_fp, train_indices, test_indices, summary_table, condition)

            #Combined descriptor models
#             X_comb = pd.concat([X_sigs, X_fp], axis=1)
#             y_comb = y_sigs

#             condition = target + '__' + cell_line + '__combined_' + str(i)
#             summary_table = rf_workflow(X_comb, y_comb, train_indices, test_indices, summary_table, condition)
                
    print('Models ' + str(i) + ' are done! Time elapsed: {} seconds'.format(time.time() - time_start))

print('Models are done! Time elapsed: {} seconds'.format(time.time() - time_start))

ABCB1__MCF7__0
Fitting 4 folds for each of 9 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:    9.6s finished


{'max_depth': 8}
Fitting 4 folds for each of 9 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:    4.5s finished


{'max_depth': 2}
ABCB1__HA1E__0
Fitting 4 folds for each of 9 candidates, totalling 36 fits


...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:    4.0s finished


{'max_depth': 3}
AR__MCF7__5
Fitting 4 folds for each of 9 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:    5.3s finished


{'max_depth': 5}


KeyboardInterrupt: 

In [22]:
summary_table.to_csv(performances_directory + 'raw_model_performances.csv')

In [23]:
grouped_table = summary_table.groupby(['target', 'cell_line', 'input']).mean()
grouped_table.to_csv(performances_directory + 'grouped_model_performances.csv')